In [348]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import sys
sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

#%run -i 'parameters'
#%run -i 'main_functions'
#%run -i 'utility_functions'

from RE_init import *
from main_functions import *
from utility_functions import *

##ALL the books 
pronoun_prefix = "_raw" # no pronoun: "_raw" - with pronoun: "0601"
file_input_postfix = "_relations_-1.csv" 
svp_prefix = "_with_SVP"

##MICE_AND_MEN
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Mice_and_men/results_with_svp_relationships/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/mice_and_men/results_with_svp_relationships/'
#file_input_name = "mice_and_men" + pronoun_prefix + file_input_postfix
#input_name_prefix = 'mice_and_men' + pronoun_prefix + svp_prefix
#DATA_SET = 'goodreads-Mice-and-men'

##HOBBIT
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Hobbit/results_with_svp_relationships/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/hobbit/results_with_svp_relationships/'
#file_input_name= 'hobbit' + pronoun_prefix + file_input_postfix
#input_name_prefix = 'hobbit' + pronoun_prefix + svp_prefix
#DATA_SET = 'goodreads-Hobbit'

##Mockingbird
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Mockingbird/results_with_svp_relationships/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/mockingbird/results_with_svp_relationships/'
#file_input_name= 'mockingbird' + pronoun_prefix + file_input_postfix
#input_name_prefix = 'mockingbird' + pronoun_prefix + svp_prefix
#DATA_SET = 'goodreads-Mockingbird'

##Frankenstein
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Frankenstein/results_with_svp_relationships/'
based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/frankenstein/results_with_svp_relationships/'
file_input_name= 'frankenstein' + pronoun_prefix + file_input_postfix
input_name_prefix = 'frankenstein' + pronoun_prefix + svp_prefix
DATA_SET = 'goodreads-Frankenstein'

def read_df_rel(based_dir, file_input_name):
    file_input = based_dir + file_input_name    
    ff = open(file_input)
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_rels = read_df_rel(based_dir, file_input_name)

In [349]:
pd.set_option('display.max_colwidth', -1)
df_rels = df_rels.replace(np.nan, '', regex=True)

In [350]:
#df = df_rels.sample(10)
def parse_rel_prepositions(rel_prep):
    res = []
    res_entry = defaultdict(list)
    prepositions = rel_prep.split("--")
    for p in prepositions:
        p = p.strip()
        if p:
            res_entry = {}
            p = p.strip()
            m = re.search('REASON:(.+?)TEXT:', p)
            if m:
                found = m.group(1).strip()
                found = found.split("_")[1:]
                prep_str = ""
                for i in found:
                    prep_str += i + " "

                prep_str = prep_str.strip()
                res_entry["prep"] = prep_str
                
            m = re.search('TEXT:(.*)$', p)
            if m:
                found = m.group(1).strip()
                res_entry["text"] = found              
            res.append(res_entry)
            
    return res

res = parse_rel_prepositions("REASON: prep_into TEXT: the time {period} --")            
print res

[{'text': 'the time {period}', 'prep': 'into'}]


In [351]:
list_rels_with_svp = []
for ind, row in df_rels.iterrows():
    #index, data = row
    list_rels_with_svp.append(row.tolist())
    if row["rel_prepositions"]:
        prepositions = parse_rel_prepositions(row["rel_prepositions"])
        for p in prepositions:
            new_row = row.copy()
            new_row["type"] = "SV(O)P"
            new_row["pattern"] = "(nsubj, verb, (O)prep)"
            new_row["rel"] = row["rel"] + " <<" + row["arg2"] + ">> " + p["prep"]
            new_row["arg2"] = p["text"]
            new_row["arg2_with_pos"] = ''
            new_row["rel_prepositions"] = ''
            list_rels_with_svp.append(new_row.tolist())

In [ ]:
df = pd.DataFrame(list_rels_with_svp,columns = df_rels.columns.tolist())
df

In [353]:
df.to_csv(based_dir + input_name_prefix + file_input_postfix,
          sep=',', encoding='utf-8',header=True, columns=df_rels.columns.tolist())

In [354]:
df2 = df[df["type"].str.contains("P")]

In [355]:
df2['rel'].value_counts()

would {recommend} <<this {book}>> to                                        14
{read} <<{this}>> in                                                        8 
{compare} <<the much vaguer {descriptions} Ingolstadt>> to                  7 
{read} <<{it}>> in                                                          6 
Otherwise apart where actually {noted} <<EVERY SINGLE PAGE {NUMBER}>> in    6 
Here How {describe} <<{emotions}>> at                                       5 
{giving} <<{life}>> to                                                      5 
Here How {describe} <<{emotions}>> about                                    5 
just {catch} <<Mr. {Wretch}>> in                                            5 
not out so {throws} <<a hissy {fit}>> about                                 5 
of {take} <<a deeper {look}>> into                                          5 
just {catch} <<Mr. {Wretch}>> on                                            5 
out {perceives} <<{himself}>> for                   

In [ ]:
entity_versions = get_entity_versions(DATA_SET)
rels_to_network(df,
                input_fname=input_name_prefix,
                output_dir_arg=based_dir,
                MAX_ITERATION=-1,
                NODE_SELECTION=True,
                DATA_SET=DATA_SET,
                SAVE_GEFX=True,
                SAVE_PAIRWISE_RELS=True,
                SHOW_ARGUMENT_GRAPH=True,
                SAVE_G_JSON=True,
                SAVE_DF_SELECTED=True)

In [357]:
df2

post_num  sentence_num  \
4      0         14             
7      0         17             
8      0         17             
10     0         19             
12     0         19             
14     0         19             
16     0         19             
18     0         19             
20     0         19             
29     0         35             
31     0         35             
35     1         2              
39     3         1              
42     4         14             
51     7         9              
72     9         36             
74     9         36             
84     11        17             
87     14        3              
101    17        27             
108    19        28             
123    22        13             
125    22        14             
132    23        19             
135    23        19             
137    24        2              
138    24        2              
140    24        2              
144    24        5              
151    24        15             
...    ..        ..             
10751  2823      3              
10754  2824      4              
10758  2825      3              
10762  2826      2              
10772  2829      7              
10785  2835      1              
10792  2837      2              
10802  2839      5              
10810  2841      5              
10823  2847      2              
10828  2850      0              
10832  2850      4              
10846  2855      3              
10856  2860      1              
10859  2861      2              
10864  2862      0              
10871  2863      1              
10875  2865      1              
10880  2865      8              
10886  2867      2              
10901  2875      0              
10906  2876      4              
10929  2885      11             
10930  2885      11             
10936  2887      1              
10939  2887      4              
10941  2887      4              
10943  2887      4              
10946  2888      4              
10954  2890      3              

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            sentence  \
4      OK, to be fair, I need to tell you what I liked about this.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
7      At 19 I was just looking for my next college boyfriend, not penning the great English classic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
8      At 19 I was just looking for my next college boyfriend, not penning the great English classic.                                                                                                           